# Interactive Cropping Tool Demo

In [4]:
# import interactive_crop
import cropping_widget
import os
image_list = ['image_samples/'+i for i in os.listdir('image_samples')]
cropping_widget.main(image_list)
# print(final_result)
# print('Finished')

GridBox(children=(Dropdown(description='Image Name:', layout=Layout(grid_area='im_selector'), options=('image_…